# How to immprove the RAG System in Llamaindex
<hr>


## This model contains four part
- Data Ingestion
- Data indexing
- Storing
- Query Interface

### Data Parsing & Ingestion
- Data -> Data parsing + Ingestion -> Index

### Data Querying
- Index -> Retrival -> LLM + Prompts -> Response

## Naive RAG - 5 Line Starter with LlamaIndex

Naive RAG Failure modes at 

**Summarization**,**Comparison**, **Implicit Data** and **Multi-part Question**.

Here's a simple 5-line starter code for LlamaIndex:

1. **Load and Parse Documents**  
   Use `SimpleDirectoryReader` to load documents from the directory:

    ```
    documents = SimpleDirectoryReader("data").load_data()
    ```

2. **Embed and Index the Documents**  
   Create a vector index using the `from_documents` method:

    ```
    index = VectorStoreIndex.from_documents(documents)
    ```

3. **Generate the Query Engine**  
   Convert the index into a queryable engine:

    ```
    query_engine = index.as_query_engine()
    ```

4. **Run Queries**  
   Use the query engine to ask questions:

    ```
    response = query_engine.query("What did the author do growing up?")
    ```

5. **Print the Response**  
   Output the response:

    ```
    print(response)
    ```

## Customization

**Ingestion**
- Ingestion Pipeline
    - transformation (sentenceSplitter(chunck_size, chunck_overlap))<br>
    ```python
    pipeline = IngestionPipeline(transformation[...])
    nodes = pipeline.run(documents)
    index = VectorStoreIndex(nodes)
    ```
    - 
    
**Retriever**
- See promts
    - `promts_dic = query_engine.get_prompts()`
    - 1. **Define the Prompt Template**  
    Create a prompt string that instructs the model to answer in Shakespearean style:<br>
        ```python
        qa_prompt_tmpl_str = (
            "Context information is below.\\n"
            "--------------------\\n"
            "{context_str}\\n"
            "--------------------\\n"
            "Given the context information and not prior knowledge, "
            "answer the query in the style of a Shakespeare play.\\n"
            "Query: {query_str}\\n"
            "Answer: "
        )
        ```

    - 2. **Create the Prompt Template Object**  
    Wrap the string into a `PromptTemplate` object:

        ```python
        qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
        ```

    - 3. **Update the Query Engine with Custom Prompts**  
    Customize the query engine to use this template for responses:

        ```python
        query_engine.update_prompts(
            {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
        )
        ```

- Configuration retirever
    - VectorIndexRetriever (similarity_top_k...) 
    - response_synthesizer = get_response_synthesizer(response_mode='tree_summarize')

## Two ways to improve RAG

1. **Data Quality**
- LlamaParse
- LlamaHub.ai


2. **Query Sophistication**
- Agentic RAG
    - Multi-turn
    - Query/task planning layer
    - Tool interface for external environment
    - Reflection
    - Memory for personalization

- 1. Routing
    - Semantic Search + Summarization
    - Router
        - Vector Query Engine (retrieve top_k)
        - Summary Query Engine (retrieve all)
- 2. Conversation memory
    - `chat_engine = index.as_chat_engine()`<br>`response = query_engine.chat("Query Quetiosn")`
- 3. Query planning
    - `query_engine = SubQuestionQueryEngine`
- 4. Tool use
    - ReAct Agent

- 3 Agent reasoning loops
    - Sequential
        - ReAct (Reason + Act)
    - DAG-based
        - Flow chart (Self-reflection)
    - Tree-based
        - come up with diffrent solution in experience

- Query Strategies
    - SubQuestionQueryEngine
    - Small-to-Big retrieval
    - Metadata filtering
    - Auto-retrieval
    - Hybrid search

- Retriever
    - RecursiveRetriever
    - SQLTableRetrieverQueryEngine




        

## Multi-document agents

- Create seperate query engines
```python
documents = SimpleDirectoryReader ("2020").load_data ( )
index2020 = VectorStoreIndex. from_documents (documents)
query_engine_2020 = index2020.as._query_engine( )

documents = SimpleDirectoryReader ("2021").load_data()
index2021 = VectorStoreIndex. from_documents (documents)
query_engine_2021 = index2021.as_query_engine()

documents = SimpleDirectoryReader ("2022" ).load_data ()
index2022 = VectorStoreIndex. from_documents (documents)
query_engine_2022 =index2022.as_query_engine()
```
- Define tools<br>
**Define the Query Engine Tools**  
   You can configure a tool with relevant metadata and description:

    ```python
    query_engine_tools = [
        QueryEngineTool(
            query_engine=query_engine_2020,
            metadata=ToolMetadata(
                name="2020_facts_tool",
                description=(
                    "Contains facts about filings "
                    "about the company from the year 2020"
                ),
            ),
        ),
        # ... etc ...
    ]
    ```
- Define agent
```python
function_llm = OpenAI(model='gpt-4o')
agent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=function_llm,
    system_prompt=f"""\
    You are a specialized agent designed to answer queries about financial data.
    You must ALWAYS use at least one of the tools provided when answering any query.
        """,

)
```

### Data Ingestion

Set-up the enviroment

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')